# MEMM + Baseline Models

In [1]:
import csv
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn import datasets
from sklearn.preprocessing import StandardScaler
from collections import Counter, defaultdict
import numpy as np

In [132]:
# Preprocessing to obtain validation counts/validation set
categories = ["B-LOC", "B-MISC", "B-ORG", "B-PER", "I-LOC", "I-MISC", "I-ORG", "I-PER", "O"]
tsv_file = open('train.txt', encoding="utf8")
tsv_reader = csv.reader(tsv_file, delimiter='\t', quotechar=None)
vocabulary = []
vocabulary2 = []
train = []
validation = []
validation_pos = []
validation_tags = []
baseline_lines = []
baseline2_lines = []
train_count_tags = defaultdict(float)
train_count_words = defaultdict(float)
train_tag_bigram = defaultdict(lambda: defaultdict(float))
val_tag_bigram = defaultdict(lambda: defaultdict(float))
train_words = defaultdict(lambda: defaultdict(float))
val_words = defaultdict(lambda: defaultdict(float))
val = False
i = 0
line = []
for row in tsv_reader:
    if i % 3 == 0:
        line = []
        if i % 15 == 0:
            val = True
        else:
            val = False
        if not val:
            for element in row:
                if element not in vocabulary2:
                    vocabulary2.append(element)
                    num2 = np.random.randint(0,100)
                    if num2 < 30:
                        line.append("<UNK>")
                        train_count_words["<UNK>"] += 1
                        continue
                train_count_words[element] += 1
                line.append(element)
            train.append(row)
        else:
            for element in row:
                line.append(element)
            validation.append(line)
    if i %3 == 1 and val:
        validation_pos.append(line)
    if i % 3 == 2:
        line_index = 0
        previous_tag = "<s>" #only adding start tags to the very first entry
        if val:
            validation_tags.append(row)
        for element in row:
            if not val:
                train_words[line[line_index]][element] += 1
                train_tag_bigram[element][previous_tag] += 1
                previous_tag = element
                train_count_tags[element] += 1
            else:
                val_words[line[line_index]][element] += 1
                val_tag_bigram[element][previous_tag] += 1
                previous_tag = element
            line_index += 1
    i += 1
for i in vocabulary2:
    if not train_count_words[i] == 0:
        vocabulary.append(i)

In [84]:
# Used to calculate word shape as a feature
def shape(word):
    word_shape = 'other'
    if re.match('[0-9]+(\.[0-9]*)?|[0-9]*\.[0-9]+$', word):
        word_shape = 'number'
    elif re.match('\W+$', word):
        word_shape = 'punct'
    elif re.match('[A-Z][a-z]+$', word):
        word_shape = 'capitalized'
    elif re.match('[A-Z]+$', word):
        word_shape = 'uppercase'
    elif re.match('[a-z]+$', word):
        word_shape = 'lowercase'
    elif re.match('[A-Z][a-z]+[A-Z][a-z]+[A-Za-z]*$', word):
        word_shape = 'camelcase'
    elif re.match('[A-Za-z]+$', word):
        word_shape = 'mixedcase'
    elif re.match('__.+__$', word):
        word_shape = 'wildcard'
    elif re.match('[A-Za-z0-9]+\.$', word):
        word_shape = 'ending-dot'
    elif re.match('[A-Za-z0-9]+\.[A-Za-z0-9\.]+\.$', word):
        word_shape = 'abbreviation'
    elif re.match('[A-Za-z0-9]+\-[A-Za-z0-9\-]+.*$', word):
        word_shape = 'contains-hyphen'
    return word_shape


In [123]:
with open('train.txt','r') as file:
        train =[x.strip().split('\t') for x in file]

In [124]:
# CONVERTING TRAINING SET TO FEATURES FOR OUR LOGISTIC REGRESSION MODEL
import re

training_x = []
training_y = []
train_set = []
for i in range(len(train)):
    if i%3 == 0 and not i%15 == 0:
        for j in range(len(train[i])):
            feature_dictionary = defaultdict(int)
            features = []
            #features.append(train[i][j])#word itself
            if j >0:
                #previous bio tag
                features.append(int(train[i+2][j-1]=='B-ORG')) 
                features.append(int(train[i+2][j-1]=='B-PER'))
                features.append(int(train[i+2][j-1]=='B-LOC'))
                features.append(int(train[i+2][j-1]=='B-MISC'))
                features.append(int(train[i+2][j-1]=='I-ORG'))
                features.append(int(train[i+2][j-1]=='I-PER'))
                features.append(int(train[i+2][j-1]=='I-LOC'))
                features.append(int(train[i+2][j-1]=='I-MISC'))
                features.append(int(train[i+2][j-1]=='O'))
                features.append(int((train[i+1][j-1])=='NNP'))# previous part of speech
                features.append(int((train[i+1][j-1])=='NNPS'))
                features.append((int((train[i+1][j-1])=='IN')))
                features.append((int((train[i+1][j-1])=='JJ')))
                features.append((int((train[i+1][j-1])=='VB')))
                features.append((int((train[i+1][j-1])=='VBD')))
                features.append(int(train[i][j-1]==','))
#                 features.append(len(train[i][j-1]))
                features.append(int(train[i][j-1][0].isupper()))#first letter capital
                features.append(int(train[i][j-1].isupper()))#whole word capitalized
                features.append(int(shape(train[i][j-1]) == 'abbreviation'))
                features.append(int(shape(train[i][j-1]) == 'ending-dot'))
                features.append(int(shape(train[i][j-1]) == 'wildcard'))
                features.append(int(shape(train[i][j-1]) == 'mixedcase'))
                features.append(int(shape(train[i][j-1]) == 'camelcase'))
                features.append(int(shape(train[i][j-1]) == 'number'))
                features.append(int(shape(train[i][j-1]) == 'punct'))
                
                
                if j < len(train[i])-1:
                    features.append(int((train[i+1][j+1])=='NNP'))# previous part of speech
                    features.append(int((train[i+1][j+1])=='NNPS'))
                    features.append((int((train[i+1][j+1])=='IN')))
                    features.append((int((train[i+1][j+1])=='JJ')))
                    features.append((int((train[i+1][j+1])=='POS')))
                    features.append((int((train[i+1][j+1])=='RB')))
                    features.append((int((train[i+1][j+1])=='VB')))
                    features.append((int((train[i+1][j+1])=='VBD')))
                    
                    features.append(int(train[i][j+1]==','))
                    features.append(int(train[i][j+1][0].isupper()))#first letter capital
                    features.append(int(train[i][j+1].isupper()))#whole word capitalized
                    features.append(int(shape(train[i][j+1]) == 'abbreviation'))
                    features.append(int(shape(train[i][j+1]) == 'ending-dot'))
                    features.append(int(shape(train[i][j+1]) == 'wildcard'))
                    features.append(int(shape(train[i][j+1]) == 'mixedcase'))
                    features.append(int(shape(train[i][j+1]) == 'camelcase'))
                    features.append(int(shape(train[i][j+1]) == 'number'))
                    features.append(int(shape(train[i][j+1]) == 'punct'))
                else:
                    for l in range(25,43):
                        features.append(0)
            else:
                for l in range (0,43):
                    features.append(0)
            features.append(int((train[i+1][j])=='NNP'))
            features.append(int((train[i+1][j])=='NNPS'))
            features.append(int(train[i][j][0].isupper()))#first letter capital
            features.append(int(train[i][j].isupper()))#whole word capitalized
            features.append(int('-' in train[i][j] and (len(train[i][j])>0)))
            features.append(len(train[i][j]))
            features.append(int(train[i][j].isalpha()))
            features.append(int(shape(train[i][j]) == 'abbreviation'))
            features.append(int(shape(train[i][j]) == 'ending-dot'))
            features.append(int(shape(train[i][j]) == 'wildcard'))
            features.append(int(shape(train[i][j]) == 'mixedcase'))
            features.append(int(shape(train[i][j]) == 'camelcase'))
            features.append(int(shape(train[i][j]) == 'number'))
            features.append(int(shape(train[i][j]) == 'punct'))
           
            tupled = feature_dictionary, train[i+2][j]
            train_set.append(tupled)
            training_x.append(features)
            training_y.append(train[i+2][j])

In [125]:
# RUNNING LOGISTIC REGRESSION CLASSIFICATION MODEL ON OUR TRAINING SET
X = training_x
y = training_y

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=1)
clf = LogisticRegression(random_state=0, multi_class='multinomial', solver='newton-cg')

model = clf.fit(X, y)
print(model.score(X_val,y_val)) # CALCULATING MODEL ACCURACY ON PRE-MADE VALIDATION SET

0.921864801865


In [126]:
# CONVERTING TEST SET TO FEATURES FOR MAXENT MODEL
with open('test.txt','r') as file:
        test =[x.strip().split('\t') for x in file]
final_preds = []

tests_x = []

testing_x = []
predictions = []
for m in range (0, len(test)):
    idk_x = []
    mini_preds = []
    semifinal_preds = []
    if m%3 == 0:
        for n in range(len(test[m])):
            test_features = []
            currentIndex = int(''.join(test[m+2]).split(' ')[0]) + n
            if n > 0:
#                 print('here')
                test_features.append(int(model.predict([testing_x[currentIndex-1]])=='B-ORG'))
                test_features.append(int(model.predict([testing_x[currentIndex-1]])=='B-PER'))
                test_features.append(int(model.predict([testing_x[currentIndex-1]])=='B-LOC'))
                test_features.append(int(model.predict([testing_x[currentIndex-1]])=='B-MISC'))
                test_features.append(int(model.predict([testing_x[currentIndex-1]])=='I-ORG'))
                test_features.append(int(model.predict([testing_x[currentIndex-1]])=='I-PER'))
                test_features.append(int(model.predict([testing_x[currentIndex-1]])=='I-LOC'))
                test_features.append(int(model.predict([testing_x[currentIndex-1]])=='I-MISC'))
                test_features.append(int(model.predict([testing_x[currentIndex-1]])=='O'))
                test_features.append(int((test[m+1][n-1])=='NNP'))
                test_features.append(int((test[m+1][n-1])=='NNPS'))
                test_features.append(int((test[m+1][n-1])=='IN'))
                test_features.append(int((test[m+1][n-1])=='JJ'))
                test_features.append((int((test[m+1][n-1])=='VB')))
                test_features.append((int((test[m+1][n-1])=='VBD')))
                test_features.append(int(test[m][n-1]==','))
#                 test_features.append(len(test[m][n-1]))
                test_features.append(int(test[m][n-1][0].isupper()))#first letter capital
                test_features.append(int(test[m][n-1].isupper()))#whole word capitalized
                test_features.append(int(shape(test[m][n-1]) == 'abbreviation'))
                test_features.append(int(shape(test[m][n-1]) == 'ending-dot'))
                test_features.append(int(shape(test[m][n-1]) == 'wildcard'))
                test_features.append(int(shape(test[m][n-1]) == 'mixedcase'))
                test_features.append(int(shape(test[m][n-1]) == 'camelcase'))
                test_features.append(int(shape(test[m][n-1]) == 'number'))
                test_features.append(int(shape(test[m][n-1]) == 'punct'))
                
                if n < len(test[m])-1:
                    test_features.append(int((test[m+1][n+1])=='NNP'))
                    test_features.append(int((test[m+1][n+1])=='NNPS'))
                    test_features.append(int((test[m+1][n+1])=='IN'))
                    test_features.append(int((test[m+1][n+1])=='JJ'))
                    test_features.append((int((test[m+1][n+1])=='POS')))
                    test_features.append((int((test[m+1][n+1])=='RB')))
                    test_features.append((int((test[m+1][n+1])=='VB')))
                    test_features.append((int((test[m+1][n+1])=='VBD')))
                    test_features.append(int(test[m][n+1]==','))
    #                 test_features.append(len(test[m][n-1]))
                    test_features.append(int(test[m][n+1][0].isupper()))#first letter capital
                    test_features.append(int(test[m][n+1].isupper()))#whole word capitalized
                    test_features.append(int(shape(test[m][n+1]) == 'abbreviation'))
                    test_features.append(int(shape(test[m][n+1]) == 'ending-dot'))
                    test_features.append(int(shape(test[m][n+1]) == 'wildcard'))
                    test_features.append(int(shape(test[m][n+1]) == 'mixedcase'))
                    test_features.append(int(shape(test[m][n+1]) == 'camelcase'))
                    test_features.append(int(shape(test[m][n+1]) == 'number'))
                    test_features.append(int(shape(test[m][n+1]) == 'punct'))
                else:
                    for i in range(25,43):
                        test_features.append(0)
            else:
                for i in range(43):
                    test_features.append(0)
                    
                    
            test_features.append(int((test[m+1][n])=='NNP'))
            test_features.append(int((test[m+1][n])=='NNPS'))
            test_features.append(int(test[m][n][0].isupper()))
            test_features.append(int(test[m][n].isupper()))
            test_features.append(int('-' in test[m][n] and (len(test[m][n])>0)))
            test_features.append(len(test[m][n]))
            test_features.append(int(test[m][n].isalpha()))
            test_features.append(int(shape(test[m][n]) == 'abbreviation'))
            test_features.append(int(shape(test[m][n]) == 'ending-dot'))
            test_features.append(int(shape(test[m][n]) == 'wildcard'))
            test_features.append(int(shape(test[m][n]) == 'mixedcase'))
            test_features.append(int(shape(test[m][n]) == 'camelcase'))
            test_features.append(int(shape(test[m][n]) == 'number'))
            test_features.append(int(shape(test[m][n]) == 'punct'))
          
#             print("test features", test_features)
            testing_x.append(test_features)
            idk_x.append(test_features)
            mini_preds.append(model.predict_proba([test_features]))
            semifinal_preds.append(model.predict([test_features]))
        tests_x.append(idk_x)
        predictions.append(mini_preds)
        final_preds.append(semifinal_preds)

In [98]:
# MEMM VITERBI ALGORITHM
def viterbi_memm_o(line, probs):
    categories = ["B-LOC", "B-MISC", "B-ORG", "B-PER", "I-LOC", "I-MISC", "I-ORG", "I-PER", "O"]
    backpointer = np.zeros([len(categories), len(line)])
    score = np.zeros([len(categories), len(line)])
    
    for i in range(len(categories)):
        score[i][0] = probs[0][0][i] 
        backpointer[i][0] = 0
        
    for t in range(1, len(line)):
        for i in range(len(categories)):
            index = 0
            saved_score = 0
            for j in range(len(categories)):
                current_score = score[j][t-1]*probs[t][0][i]
                if current_score > saved_score:
                    saved_score = current_score
                    index = j
            score[i][t] = saved_score
            backpointer[i][t] = index
    sequence = np.zeros(len(line))
    max_s = 0
    max_index = 0
    for i in range(9):
        if max_s < score[i][-1]:
            max_s = score[i][-1]
            max_index = i
    sequence[-1] = max_index
    char_sequence = []
    for i in range(len(line)-2, -1, -1):
        sequence[i] = backpointer[int(sequence[i+1])][i+1]
    for i in range(len(line)):
        char_sequence.append(categories[int(sequence[i])])
    return char_sequence

In [227]:
# Going through test set and executing our MEMM Viterbi algorithm on the set
tsv_file = open('test.txt', encoding="utf8")
tsv_reader = csv.reader(tsv_file, delimiter='\t', quotechar=None)
PER = []
LOC = []
ORG = []
MISC = []
indices = []
tags = []
baseline2_display = []
baseline1_display = []
display = []
i = 0
for row in tsv_reader:
    if i%3 == 0:
        index = int(i/3)

        line_tags = viterbi_memm_o(row, predictions[index])
        baseline1_display.append(baseline(row))
        baseline2_display.append(baseline2(row))

        display.append(line_tags)
        for tag in line_tags:
            tags.append(tag)
    i+=1
tsv_file = open('test.txt', encoding="utf8")
i = 0
tsv_reader = csv.reader(tsv_file, delimiter=' ', quotechar=None)
for row in tsv_reader:
    if i%3 == 2:
        for element in row:
            indices.append(element)
    i+= 1
for i in range(len(tags)):
    if "PER" in tags[i]:
         tags[i] = "PER"
    elif "LOC" in  tags[i]:
         tags[i] = "LOC"
    elif "ORG" in  tags[i]:
         tags[i] = "ORG"
    elif "MISC" in  tags[i]:
         tags[i] = "MISC"
    else:
         tags[i] = "O"
beginning = False
log_tag = ""
end = ""
begin = ""
intermediate = False
for i in range(len(tags)):
    if beginning and log_tag == tags[i]:
        end = str(indices[i])
    if beginning and not log_tag == tags[i]:
        beginning = False
        if "PER" in log_tag:
            PER.append(begin+end)
        elif "LOC" in log_tag:
            LOC.append(begin+end)
        elif "ORG" in log_tag:
            ORG.append(begin+end)
        elif "MISC" in log_tag:
            MISC.append(begin+end)
    if beginning == False and not tags[i] == 'O':
        beginning = True
        log_tag = tags[i]
        begin = str(indices[i]) + "-"
        end = str(indices[i])

In [128]:
#WRITING PREDICTIONS TO CSV FOR SUBMISSION
csv_per = ""
csv_loc = ""
csv_org = ""
csv_misc = ""
for i in PER:
    csv_per += i + " "
for i in LOC:
    csv_loc += i + " "
for i in ORG:
    csv_org += i + " "
for i in MISC:
    csv_misc += i + " "
with open('preds_memm.csv', 'w', newline='') as csvfile:
    spamwriter = csv.writer(csvfile)
    spamwriter.writerow(["Type", "Prediction"])
    spamwriter.writerow(["PER", csv_per])
    spamwriter.writerow(["LOC", csv_loc])
    spamwriter.writerow(["ORG", csv_org])
    spamwriter.writerow(["MISC", csv_misc])

# MEMM Validation + Baseline Experiments

In [134]:
#Converting validation set to feature vectors
validating_x = []
validating_y = []
predictions_val = []
for m in range (len(validation)):
    mini_preds = []
    for n in range(len(validation[m])):
        currentIndex = n
        test_features = []
        if n > 0:
#                 print('here')
            test_features.append(int(validation_tags[m][n-1] =='B-ORG'))
            test_features.append(int(validation_tags[m][n-1]=='B-PER'))
            test_features.append(int(validation_tags[m][n-1]=='B-LOC'))
            test_features.append(int(validation_tags[m][n-1]=='B-MISC'))
            test_features.append(int(validation_tags[m][n-1]=='I-ORG'))
            test_features.append(int(validation_tags[m][n-1]=='I-PER'))
            test_features.append(int(validation_tags[m][n-1]=='I-LOC'))
            test_features.append(int(validation_tags[m][n-1]=='I-MISC'))
            test_features.append(int(validation_tags[m][n-1]=='O'))
            test_features.append(int((validation_pos[m][n-1])=='NNP'))
            test_features.append(int((validation_pos[m][n-1])=='NNPS'))
            test_features.append(int((validation_pos[m][n-1])=='IN'))
            test_features.append(int((validation_pos[m][n-1])=='JJ'))
            test_features.append((int((validation_pos[m][n-1])=='VB')))
            test_features.append((int((validation_pos[m][n-1])=='VBD')))
            test_features.append(int(validation[m][n-1]==','))
#                 test_features.append(len(test[m][n-1]))
            test_features.append(int(validation[m][n-1][0].isupper()))#first letter capital
            test_features.append(int(validation[m][n-1].isupper()))#whole word capitalized
            test_features.append(int(shape(validation[m][n-1]) == 'abbreviation'))
            test_features.append(int(shape(validation[m][n-1]) == 'ending-dot'))
            test_features.append(int(shape(validation[m][n-1]) == 'wildcard'))
            test_features.append(int(shape(validation[m][n-1]) == 'mixedcase'))
            test_features.append(int(shape(validation[m][n-1]) == 'camelcase'))
            test_features.append(int(shape(validation[m][n-1]) == 'number'))
            test_features.append(int(shape(validation[m][n-1]) == 'punct'))
            
            if n < len(validation[m])-1:
                test_features.append(int((validation_pos[m][n+1])=='NNP'))
                test_features.append(int((validation_pos[m][n+1])=='NNPS'))
                test_features.append(int((validation_pos[m][n+1])=='IN'))
                test_features.append(int((validation_pos[m][n+1])=='JJ'))
                test_features.append((int((validation_pos[m][n+1])=='POS')))
                test_features.append((int((validation_pos[m][n+1])=='RB')))
                test_features.append((int((validation_pos[m][n+1])=='VB')))
                test_features.append((int((validation_pos[m][n+1])=='VBD')))
                test_features.append(int(validation[m][n+1]==','))
#                 test_features.append(len(test[m][n-1]))
                test_features.append(int(validation[m][n+1][0].isupper()))#first letter capital
                test_features.append(int(validation[m][n+1].isupper()))#whole word capitalized
                test_features.append(int(shape(validation[m][n+1]) == 'abbreviation'))
                test_features.append(int(shape(validation[m][n+1]) == 'ending-dot'))
                test_features.append(int(shape(validation[m][n+1]) == 'wildcard'))
                test_features.append(int(shape(validation[m][n+1]) == 'mixedcase'))
                test_features.append(int(shape(validation[m][n+1]) == 'camelcase'))
                test_features.append(int(shape(validation[m][n+1]) == 'number'))
                test_features.append(int(shape(validation[m][n+1]) == 'punct'))
            else:
                for i in range(25,43):
                    test_features.append(0)
        else:
            for i in range(43):
                test_features.append(0)


        test_features.append(int((validation_pos[m][n])=='NNP'))
        test_features.append(int((validation_pos[m][n])=='NNPS'))
        test_features.append(int(validation[m][n][0].isupper()))
        test_features.append(int(validation[m][n].isupper()))
        test_features.append(int('-' in validation[m][n] and (len(validation[m][n])>0)))
        test_features.append(len(validation[m][n]))
        test_features.append(int(validation[m][n].isalpha()))
        test_features.append(int(shape(validation[m][n]) == 'abbreviation'))
        test_features.append(int(shape(validation[m][n]) == 'ending-dot'))
        test_features.append(int(shape(validation[m][n]) == 'wildcard'))
        test_features.append(int(shape(validation[m][n]) == 'mixedcase'))
        test_features.append(int(shape(validation[m][n]) == 'camelcase'))
        test_features.append(int(shape(validation[m][n]) == 'number'))
        test_features.append(int(shape(validation[m][n]) == 'punct'))


#             print("test features", test_features)
        validating_x.append(test_features)
        validating_y.append(validation_tags[m][n])
        mini_preds.append(model.predict_proba([test_features]))

    predictions_val.append(mini_preds)



            

In [135]:
# First and Second Baseline Models from HMM File
def baseline(line):
    tags = []
    for element in line:
        if element not in vocabulary:
            element = "<UNK>"
        if element == "<UNK>":
            tags.append("O")
        else:
            vals = train_words[element]
            most_likely = max(vals, key=train_words[element].get)
            tags.append(most_likely)
    return tags

def baseline2(line):
    tags = []
    for element in line:
        if element not in vocabulary:
            element = "<UNK>"
        if element == "<UNK>":
            tags.append("O")
        else:
            tags.append(np.random.choice(list(train_words[element])))
    return tags

In [150]:
# Checking Precision, Recall, F-Score on validation set
memm_viterbi = []
baseline1_val = []
baseline2_val = []
def check_val(method,validation, validation_tags):
    correct = 0
    guessed = 0
    total = 0
    for i in range(len(validation)):
        tags = []
        if method == 'viterbi':
            guess_line = viterbi_memm_o(validation[i], predictions_val[i])
            memm_viterbi.append(guess_line)
        elif method == 'baseline1':
            guess_line = baseline(validation[i])
            baseline1_val.append(guess_line)
        else:
            guess_line = baseline2(validation[i])
            baseline2_val.append(guess_line)
        for tag in range(len(guess_line)):
            if "PER" in guess_line[tag]:
                tags.append("PER")
            elif "LOC" in guess_line[tag]:
                tags.append("LOC")
            elif "ORG" in guess_line[tag]:
                tags.append("ORG")
            elif "MISC" in guess_line[tag]:
                tags.append("MISC")
            else:
                tags.append("O")
        for tag in range(len(validation_tags[i])):
            if "PER" in validation_tags[i][tag]:
                validation_tags[i][tag] = "PER"
            elif "LOC" in validation_tags[i][tag]:
                validation_tags[i][tag] = "LOC"
            elif "ORG" in validation_tags[i][tag]:
                validation_tags[i][tag] = "ORG"
            elif "MISC" in validation_tags[i][tag]:
                validation_tags[i][tag] = "MISC"
            else:
                validation_tags[i][tag] = "O"
#         print(tags, validation_tags[i])
        for tag in range(len(validation_tags[i])):
            if tags[tag] == "O":
                continue
#             print(tags[tag] == validation_tags[i][tag])
            correct += (tags[tag] == validation_tags[i][tag])
        for j in guess_line:
            if not j == 'O':
                guessed += 1
        for k in validation_tags[i]:
            if not k == 'O':
                total += 1
    precision = correct/guessed
    recall = correct/total
    fscore = 2*precision*recall/(precision+recall)
    return precision, recall, fscore

In [151]:
precision, recall, fscore = check_val('viterbi', validation, validation_tags)
print(precision, recall, fscore)

0.5407725321888412 0.3152317880794702 0.3982893268873188


In [152]:
precision, recall, fscore = check_val('baseline1', validation, validation_tags)
print(precision, recall, fscore)

0.8679379789698806 0.7167034584253127 0.7851039819442205


In [153]:
precision, recall, fscore = check_val('baseline2', validation, validation_tags)
print(precision, recall, fscore)

0.32853621690387214 0.6080941869021339 0.4265950856906876


In [313]:
print(display[45])

['B-PER', 'I-PER', 'O', 'O', 'O', 'O', 'O', 'O', 'O']


In [314]:
print(test[135])

['West', 'Ham', '3', '1', '1', '1', '3', '4', '4']


In [315]:
print(baseline1_display[45])

['B-LOC', 'I-ORG', 'O', 'O', 'O', 'O', 'O', 'O', 'O']


In [244]:
print(baseline2_display[0])

['O', 'O', 'O', 'O', 'B-ORG', 'O', 'O', 'O', 'I-ORG']
